## Callin Switzer
Update 10 Feb 2017

Snippet of code to plot digitized points over the top of a video

In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%qtconsole

In [3]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [4]:
# adjust gamma
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [5]:
# choose video
# 20150608_135726.avi
aa = file_choose()

In [11]:
print aa
cap = cv2.VideoCapture(aa)
numFr = int(round(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)))
print 'numFrames = ' + str(numFr)

/Volumes/GMD/June8ArbVids/20150608_135726.avi
numFrames = 1489


In [17]:
# change directory
newD = '/Users/callinswitzer/Dropbox/ExperSummer2016/Kalmia/DigitizedPollenVid_tail'
if not os.path.isdir(newD):
    os.mkdir(newD)
    
os.chdir(newD)

In [18]:
# get digitized points

ptsFile = '/Users/callinswitzer/Desktop/DigKalmPts.csv'
digPts = pd.read_csv(ptsFile)

# show frames with points digitized
digPts.dropna(thresh = 1).head()


,anthx,anthy,polx,poly,anthx.1,anthy.1,polx.1,poly.1
0,NaN,NaN,125.293407,23.838134,NaN,NaN,122.871563,17.696408
1,NaN,NaN,125.503205,23.838134,NaN,NaN,123.475060,21.945242
2,127.218343,23.391968,125.293407,24.677323,126.712948,21.527529,124.114798,26.278108
3,126.829895,24.557313,125.293407,25.936108,126.123835,26.434469,124.839422,30.783905
4,125.664550,30.966707,123.824825,32.020233,125.748434,31.874009,125.718039,35.520649


In [22]:
# plot points on each frame of video

cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 

frameStart = 273 # first frame that anything is digitized -- looked up in file


while(ctr <= 650):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()

    if ret and ctr  > frameStart: # if we can return a frame, and it's over a certain frame number
        # our operation on frame comes here
        trueCol = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # adjust gamma
        trueCol = adjust_gamma(trueCol, 2)
        
        plt.clf()

        # Display the resulting frame
        io.imshow(trueCol)
        
        # add points
        plt.axis([0, frame.shape[1], frame.shape[0], 0])
        plt.axis('off')
        
        plt.gca().set_position([0,0,1,1])
        plt.gca().set_aspect('equal')
        
        # 0:ctr - frameStart puts a tail on the poings
        plt.scatter(x = digPts.ix[0:ctr - frameStart][[4,6]], 
            y = trueCol.shape[0]-digPts.ix[0:ctr - frameStart][[5,7]], 
           s = 50, 
           alpha = 0.5, edgecolor =  '', 
           color = ['#deebf7'])
        
        # save figure
        plt.savefig(str(ctr - frameStart).zfill(4) + '.png')
        
        
    print ctr
    ctr +=1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
# use ffmpeg to convert directory to video
stta = time.time()
!ffmpeg -start_number 0 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y digPollVid_tail.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:15.08, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 800x600 [SAR 3937:3937 DAR 4:3], 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264

In [64]:
# delete all png files in directory
filelist = [ f for f in os.listdir(".") if f.endswith(".png") ]

for f in filelist:
    os.remove(f)

In [17]:
# save section of video without points
# plot points on each frame of video

cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 


while(ctr <= 600):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()

    if ret and ctr > 500: # if we can return a frame, and it's over a certain frame number
        # our operation on frame come here
        #trueCol = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # adjust gamma
        trueCol = adjust_gamma(frame, 2)
        
        # save frame
        cv2.imwrite(str(ctr).zfill(4) + '.png', trueCol)
        
    print ctr
    ctr +=1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
stta = time.time()
!ffmpeg -start_number 500 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y unAnnotatedVid2.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:04.00, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgb24(pc), 1280x1024, 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264 @ 0x7ff604001e00] us

In [19]:
# delete all png files in directory
filelist = [ f for f in os.listdir(".") if f.endswith(".png") ]

for f in filelist:
    os.remove(f)

In [2]:
# print system info
import IPython
print IPython.sys_info()

{'commit_hash': u'5c9c918',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/Users/callinswitzer/anaconda/lib/python2.7/site-packages/IPython',
 'ipython_version': '5.1.0',
 'os_name': 'posix',
 'platform': 'Darwin-15.6.0-x86_64-i386-64bit',
 'sys_executable': '/Users/callinswitzer/anaconda/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '2.7.9 |Anaconda custom (x86_64)| (default, Dec 15 2014, 10:37:34) \n[GCC 4.2.1 (Apple Inc. build 5577)]'}


In [1]:
# show installed packages and versions
!pip freeze 

alabaster==0.7.9
anaconda-clean==1.0
anaconda-client==1.5.1
anaconda-navigator==1.3.1
appnope==0.1.0
appscript==1.0.1
argcomplete==1.0.0
astroid==1.4.7
astropy==1.2.1
Babel==2.3.4
backports-abc==0.4
backports.shutil-get-terminal-size==1.0.0
backports.ssl-match-hostname==3.4.0.2
beautifulsoup4==4.5.1
bitarray==0.8.1
blaze==0.10.1
bokeh==0.12.2
boto==2.42.0
Bottleneck==1.1.0
cdecimal==2.3
cffi==1.7.0
chest==0.2.3
click==6.6
cloudpickle==0.2.1
clyent==1.2.2
colorama==0.3.7
conda==4.2.11
conda-build==2.0.2
configobj==5.0.6
configparser==3.5.0
contextlib2==0.5.3
cryptography==1.3.1
cycler==0.10.0
Cython==0.24.1
cytoolz==0.8.0
dask==0.11.0
datashape==0.5.2
decorator==4.0.10
dill==0.2.5
docutils==0.12
dynd==0.7.3.dev1
enum34==1.1.6
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.6
Flask==0.11.1
Flask-Cors==2.1.2
funcsigs==1.0.2
functools32==3.2.3.post2
futures==3.0.5
gevent==1.1.2
greenlet==0.4.10
grin==1.2.1
h5py==2.6.0
HeapDict==1.0.0
html5lib==0.999
idna==2.1
imagesize==0.7.1
ipaddress==1